## Problem:

There is a building with `F` floors and `S` glass cups of equal durability. If you drop a cup from floor `N`:
- from floor `N` or higher — it **breaks**,
- from any lower floor — it **does not break**.

**Task:** find `N` using **as few drops as possible** in the **worst-case scenario**

## What is the "worst-case scenario"?
This is the case when the cup:
- breaks only on the highest tested floor,
- or doesn't break for a long time,
- or breaks very early, requiring a manual search.

Goal: **minimize the maximum number of drops in any scenario**.

## Why is a fixed-step strategy ineffective?

Let the step be 10 floors: 10 → 20 → 30 → 40 → 50 (it breaks :( )
- from floor `50` or higher — it **breaks**, and now we have -1 cup
- only 1 cup remains, we must be careful — no extra cups. We now check floors `41` - `49`
That gives: 10, 20, 30, 40 (didn’t break), 41, 42, 43, 44, 45, 46, 47, 48, 49, 50 (breaks) = `14` drops for `2` cups — not bad.

But this method ignores the worst-case where:
- Suppose the cup breaks on floor `100`, and then the number of drops becomes `19`.

**We are wasting too many unnecessary drops with the first cup:**

- The first cup is a very valuable resource.
And “10, 20, 30…” uses too many tries on large jumps, without considering how many are left.

- The step doesn’t adapt to the remaining floors. When we reach the 90th floor, there are only 10 left. But the step is still 10.
This is inefficient — the risk of breaking is high, but the gain is small.

So: the fixed-step strategy (like “10, 20, 30…”) is not optimal,
because it doesn’t guarantee the minimum number of drops in the worst-case
and wastes the first cup. A much better option is to intelligently reduce the steps to balance speed and caution.

## So what strategy is optimal, considering all risks and using cups wisely?

First cup — **optimized jump search**
- We throw the cup in **large jumps**, for example: 14, 27, 39, 50...
- Each next jump is **smaller** than the previous one (14, 13, 12, ...).
- This is based on the triangular number:
  $$
  1 + 2 + 3 + \dots + k = \frac{k(k + 1)}{2} \geq F
  $$
- We find the minimum `k` for which this sum >= `F` floors.

This allows us to **narrow down the search range as much as possible**.

Second cup — **linear search**
Why not jump with it too?

•	Only one cup remains.  
•	If it breaks — we might never find the answer.

- When the first cup breaks at floor `P`, the previous drop was at `Q`, so `N` is between `Q+1` and `P-1`.
- We test **each floor in order using the second cup**, because we **can’t risk it**.

Why is this the minimum?
- The formula accounts for **all possible outcomes** (breaks/doesn't break).
- Each drop **reduces the search range** in both directions.
- We are looking for the minimum `k` such that **we cover all `F` floors**.

So, no matter how bad the case is — we will **definitely find the answer in `k` drops**.


## How to implement this in code:

We’ll use a recurrence formula:

$$
f(k, s) = f(k - 1, s - 1) + f(k - 1, s) + 1
$$

This means:
- If the cup breaks = fewer cups
- If it doesn't break = same number of cups, fewer attempts
- `+1` — the floor we're currently testing

In [1]:
import numpy as np

def min_drops(floors, glasses):
    # dp[g] — number of floors we can test with g glasses and current number of drops
    dp = np.zeros(glasses + 1, dtype=int)
    drops = 0

    while dp[glasses] < floors:
        drops += 1
        for g in range(glasses, 0, -1):
            dp[g] = dp[g] + dp[g - 1] + 1
    return drops

# Run the program
if __name__ == "__main__":
    floors = int(input("Enter the number of floors: "))
    glasses = int(input("Enter the number of glasses: "))
    result = min_drops(floors, glasses)
    print(f"Number of floors: {floors}")
    print(f"Number of glasses: {glasses}")
    print(f"Minimum number of drops in the worst case: {result}")

Number of floors: 100
Number of glasses: 2
Minimum number of drops in the worst case: 14


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

def generate_dp_table(floors, glasses):
    """
    f(k, s) — the maximum number of floors we can check
    using k drops and s glasses (dynamic programming).
    """
    f = np.zeros(glasses + 1, dtype=int)  # f[s] = f(k, s)
    history = []

    drops = 0  # k — number of drops
    while f[glasses] < floors:
        drops += 1
        for s in range(glasses, 0, -1):
            f[s] = f[s] + f[s - 1] + 1  # f(k, s) = f(k-1, s-1) + f(k-1, s) + 1
        history.append(f.copy())

    # Table with column names and index starting from 1
    columns = [f"{s} glass(es) (s={s})" for s in reversed(range(len(f)))]
    data = [row[::-1] for row in history]
    table = pd.DataFrame(data, columns=columns)
    table.index = np.arange(1, len(table) + 1)  # start from 1
    table.index.name = "Drop # (k)"
    return drops, table

def plot_dp_graph(table, floors, glasses, drops):
    plt.figure(figsize=(10, 6))
    for s in range(glasses, 0, -1):
        col = f"{s} glass(es) (s={s})"
        plt.plot(table.index, table[col], marker='o', label=col)

    plt.axhline(y=floors, color='red', linestyle='--', label=f"Target: {floors} floors (F)")
    plt.title(f"{glasses} glass(es) (s={glasses}), {floors} floors (F)\nMinimum {drops} drops (k)")
    plt.xlabel("Drop # (k)")
    plt.ylabel("f(k, s): Max tested floors")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Interactive widgets
floors_input = widgets.Text(value='100', description='Floors (F):')
glasses_input = widgets.Text(value='2', description='Glasses (s):')
run_button = widgets.Button(description="Calculate", button_style='success')
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        try:
            floors = int(floors_input.value)
            glasses = int(glasses_input.value)

            if floors <= 0 or glasses <= 0:
                print("Please enter positive integers.")
                return

            drops, table = generate_dp_table(floors, glasses)
            print(f"Minimum number of drops (k): {drops}")
            display(table)
            plot_dp_graph(table, floors, glasses, drops)

        except ValueError:
            print("Please enter valid integers.")

run_button.on_click(on_button_clicked)

# UI display
display(widgets.VBox([floors_input, glasses_input, run_button, output]))